In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote

In [3]:

# Input Search Query
SEARCH_QUERY = 'tv'

In [4]:
PROXY_CRAWL_TOKEN = 'JNsGFIUa5_amNHUfiLkHPA'
# PROXY_CRAWL_TOKEN = 'K-9nDJsiR1tb0vbXVAaSOQ'

In [5]:

def get_proxy(url):
    quoted_url = quote(url)
    res = requests.get(f'https://api.proxycrawl.com/?token={PROXY_CRAWL_TOKEN}&url={quoted_url}')
    res.raise_for_status() # Raise error if it fails
    return res.content


In [43]:

#### A function to get the content of the page of required query
def search_in_amazon(search_query):
    url = f"https://www.amazon.com/s?k={search_query}"
    return get_proxy(url)

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)
def search_asin(asin):
    url = f"https://www.amazon.com/dp/{asin}"
    return get_proxy(url)

#### A function to pass on the link of 'see all reviews' and extract the content
def search_reviews(review_link):
    url = f"https://www.amazon.com{review_link}"
    return get_proxy(url)


### Product Name extraction

In [44]:
print("Start to extract product names")

product_names = []
data_asin = []
LAST_PAGE = 21
for i in range(1,LAST_PAGE):
    print(f"Iteration {i}/{LAST_PAGE}")
    html = search_in_amazon(SEARCH_QUERY+'&page='+str(i))
    soup = BeautifulSoup(html, 'html5lib')
    items = soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'})
    if len(items) == 0:
        print(f"Warning: No product name found in this page")
    for i in items:
        product_names.append(i.text) # adding the product names to the list

    for i in soup.findAll("div", {"class":"s-result-item"}):
        if i['data-asin']:
            data_asin.append(i['data-asin'])

print(f"Finished: {len(product_names)} product names found")

Start to extract product names
Iteration 1/21
Iteration 2/21
Iteration 3/21
Iteration 4/21
Iteration 5/21
Iteration 6/21
Iteration 7/21
Iteration 8/21
Iteration 9/21
Iteration 10/21
Iteration 11/21
Iteration 12/21
Iteration 13/21
Iteration 14/21
Iteration 15/21
Iteration 16/21
Iteration 17/21
Iteration 18/21
Iteration 19/21
Iteration 20/21
Finished: 306 product names found


In [52]:
len(product_names)

306

When scrawling the all pages of product list in specific search query, I could discover that there are same products in the list. <br>
Therefore, I needed to remove the same product in the list of ASIN.

In [53]:
print("Start to search asin")

Start to search asin


In [58]:
links = []

In [ ]:

data_asin = list(set(data_asin)) # Leave unique values only

for i in range(len(data_asin)):
    print(f"Searching asin {i}/{len(data_asin)}")
    html = search_asin(data_asin[i])
    soup = BeautifulSoup(html, 'html5lib')
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        links.append(i['href'])

Searching asin 0/273
Searching asin 1/273
Searching asin 2/273
Searching asin 3/273
Searching asin 4/273
Searching asin 5/273
Searching asin 6/273
Searching asin 7/273
Searching asin 8/273
Searching asin 9/273
Searching asin 10/273
Searching asin 11/273
Searching asin 12/273
Searching asin 13/273
Searching asin 14/273
Searching asin 15/273
Searching asin 16/273
Searching asin 17/273
Searching asin 18/273
Searching asin 19/273
Searching asin 20/273
Searching asin 21/273
Searching asin 22/273
Searching asin 23/273
Searching asin 24/273
Searching asin 25/273
Searching asin 26/273
Searching asin 27/273
Searching asin 28/273
Searching asin 29/273
Searching asin 30/273
Searching asin 31/273
Searching asin 32/273
Searching asin 33/273
Searching asin 34/273
Searching asin 35/273
Searching asin 36/273
Searching asin 37/273
Searching asin 38/273


In [57]:
len(links)

0

In [11]:
links = list(set(links))
print(f"Finished: {len(links)} links found")
# The number of link and the number of ASIN can be different, because there are many products which have no review.

Finished: 1 links found


In [51]:

print("Start to search reviews")
# urls=[]
# titles = []
search_query_list = []
reviews=[]
ratings=[]
# dates=[]



Start to search reviews


In [50]:

print("Start to search reviews")


for j in range(0, len(links)):

        target = links[j]
        print(f"Searching reviews: {j}/{len(links)}, target is {target}")
        for k in range(1, 50000):
            html = search_reviews(f"{target}&pageNumber={k}")
            soup = BeautifulSoup(html, 'html5lib')
            if soup.find(
                'div',
                {"class" : "a-section a-spacing-top-large a-text-center no-reviews-section"}
            ):
                print(f'Page {k}: No more reviews, step to next link')
                break
            else:
#                 items = soup.select('span[data-hook="review-body"] > span')
                items = soup.select('div[data-hook="review"]')
#                 reviews_in_page = soup.select()
#                 ratings_in_page = soup.select('div.reviews-content span.a-icon-alt')
        
                prev_reviews_cnt = len(reviews)
                prev_ratings_cnt = len(ratings)
                
                for i in items:
                    ratings_in_page = i.find("i", {'data-hook':'review-star-rating'}).text
                    reviews_in_page = i.find('span', {'data-hook': 'review-body'}).text
                    if reviews_in_page == '':
                        # Ignore blank elements
                        continue
                    else:
                        ratings.append(ratings_in_page)
                        reviews.append(reviews_in_page)
                        search_query_list.append(SEARCH_QUERY)
                        
                
                print(f"Page {k}: {len(ratings) - prev_ratings_cnt} ratings found")
                print(f"Page {k}: {len(reviews) - prev_reviews_cnt} reviews found")
                print(f"current number of ratings : {len(ratings)}")
                print(f"current number of reviews : {len(reviews)}")
                if len(ratings) != len(reviews):
                    print("something error")
                    sys.exit(1)
                print("---------------------------")

                
                
print(f"Finished: {len(reviews)} reviews found")


Start to search reviews
Searching reviews: 0/1, target is /VIZIO-Class-49-5-Diag-Smart/product-reviews/B07KJB92L9?reviewerType=all_reviews
Page 1: 10 ratings found
Page 1: 10 reviews found
current number of ratings : 10
current number of reviews : 40
something error


NameError: name 'sys' is not defined

In [40]:
print(reviews)
print(ratings)

["\n\n\n\n\n\n\n\n  \n  \n    Bought this from Costco after trying three other sets that didn't have great pictures. Got this home and set up in under an hour, The settings are pretty extensive and allows for a lot of tweeking. Normal reception (from Comcast cable) is extremely limited by 1080P and picture is not as sharp as it could be. Since we don't subscribe to streaming services that provide 4K, can't really comment on extended viewing of 4K content. Vizio does have a short 4K video that can be downloaded to a thumb drive and plugged into the USB port.  The picture from the video is outstanding. It is crisp, clear and sharp. The sound is a little low, but a sound bar alleviates this issue. After trying LG, Hisense and another Vizio (developed a vertical line after 4 days), we went with this Vizio model and are pretty pleased so far. Plus, Costco gives you and extra year of warranty at no cost. I would recommend this TV.\n  \n", '\n\n\n\n\n\n\n\n  \n  \n    I purchased a unit from 

In [42]:
print(len(reviews))
print(len(ratings))

10
10


In [202]:
reviews[-15:]

['This is my second Vizio TV I had bought. They have great picture and very quality tv ! Customer service is great in case you ever have to use it!',
 "Bought the renewed one to save a bit of money, turned out to be packaged and looked brand new. Couldn't tell the differenceArrived in good condition, great TV and vizio makes a good product. Would buy again",
 'Overall the TV has worked to spec, I will most likely be ordering another one this way.',
 'I can’t say enough about this TV. All the claims are true. Wonderful picture, easy to use .',
 'When I received the television I tried attaching the legs for the tv to the tv; the screws that came with the product were too short and were not the correct screws to attach the legs to the television.',
 "I purchased a television from this merchant and it stopped working right after the Amazon 90 day warranty expired. They refused to replace or repair it which means I actually rented this item. Don't be conned as I was. Stay away from items be

In [191]:
print(reviews[-10:])
print(ratings[-10:])

["Horrible sound! The voices sounded like they had a buzz and couldn't understand the words.", 'no writing review.', 'Great tv. Slim and works great. Only down side is a slightly long boot time.', 'Nice picture', 'exactly what I wanted. I have had the same TV for 7 years and is still in use. Good for RCA', 'Picture was good but speakers had a lot of static. Returned it', 'Great  TV', 'This RCA gives a cool, crisp picture that really pops. And the 32" screen is a very good size - not obstrusive, and not a small stand-in for the real thing either. True enough, the sound is nearly a felony; especially at a higher volumn, and the convenient disk player is not blu-ray, but all together this unit offers quite a lot and we are quite satisfied.', "Very LOW quality TV, sound is of horrible quality sounding like a $5 speaker. Picture is blurry and color is horrible, worse overall than any tube TV I've seen. I made sure everything was connected right, and all settings were correct, just a very lo

In [188]:
reviews[-8:]

['Exactly a second described. Packaged well',
 'My dad has this TV. It is a 32 inch. It has a built in DVD player as well.',
 'TV stopped working properly 2 months after I bought it.',
 "Horrible sound! The voices sounded like they had a buzz and couldn't understand the words.",
 'no writing review.',
 'Great tv. Slim and works great. Only down side is a slightly long boot time.',
 'Nice picture',
 'exactly what I wanted. I have had the same TV for 7 years and is still in use. Good for RCA']

In [163]:
reviews[-7]

"Horrible sound! The voices sounded like they had a buzz and couldn't understand the words."

In [165]:
reviews[-6]='no writing review.'

In [187]:
print(reviews[-13:])
print(ratings[-13:])

['From what I have seen so far, I like it.  I am not usually critical of things so I feel the picture quality is good. I have not had it up long.  It is not a primary tv, so it is not watched except on weekends', 'Nothing', 'Easy set up. Great picture. Perfect size. Would purchase again.', 'Exceeded Expectations!', 'Suitable for small room', 'Exactly a second described. Packaged well', 'My dad has this TV. It is a 32 inch. It has a built in DVD player as well.', 'TV stopped working properly 2 months after I bought it.', "Horrible sound! The voices sounded like they had a buzz and couldn't understand the words.", 'no writing review.', 'Great tv. Slim and works great. Only down side is a slightly long boot time.', 'Nice picture', 'exactly what I wanted. I have had the same TV for 7 years and is still in use. Good for RCA']
['3.0 out of 5 stars', '5.0 out of 5 stars', '5.0 out of 5 stars', '5.0 out of 5 stars', '5.0 out of 5 stars', '5.0 out of 5 stars', '4.0 out of 5 stars', '1.0 out of 

In [111]:
ratings

['5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '1.0 out of 5 stars',
 '2.0 out of 5 stars',
 '1.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '1.0 out of 5 stars',
 '1.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '2.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '4.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '3.0 out of 5 stars',
 '1.0 out of 5 stars',
 '1.0 out of 5 stars',
 '4.0 out of 5 stars',
 '1.0 out of 5 stars',
 '5.0 out of 5 stars',
 '2.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out o

In [56]:
reviews_copy=reviews
ratings_copy=ratings

In [57]:
len(reviews_copy)

60848

In [ ]:
print("Start to generate report")
# rev={'dates':dates, 'titles':titles, 'ratings':ratings, 'reviews':reviews, 'url':urls} #converting the reviews list into a dictionary

rev={'search_query':search_query_list, 'reviews' :reviews} #converting the reviews list into a dictionary
review_data=pd.DataFrame.from_dict(rev) #converting this dictionary into a dataframe

df = review_data.replace('\n','', regex=True)

writer = pd.ExcelWriter(SEARCH_QUERY+'_review.xlsx')
df.to_excel(writer, 'Sheet1', index=False)
writer.save()
print("Success")